In [33]:
import pandas as pd
import numpy as np
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras import Sequential
from keras.layers import Flatten, Dense
from math import ceil
from tensorflow.keras.models import load_model

import datetime


Laoding data set 

In [34]:
Train_data_mask = r"C:\Users\rana ahmed\Documents\Sync intern\Task2-face\New combine Face Mask Dataset\Train"
Test_data =r"C:\Users\rana ahmed\Documents\Sync intern\Task2-face\New combine Face Mask Dataset\Test"
Validate_data = r"C:\Users\rana ahmed\Documents\Sync intern\Task2-face\New combine Face Mask Dataset\Validation"


Appling data augmentation

In [44]:
train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True, zoom_range=0.2,shear_range=0.2)
train_data=train_datagen.flow_from_directory(directory=Train_data_mask,target_size=(128,128), class_mode='categorical',batch_size=32)

val_datagen = ImageDataGenerator(rescale=1.0/255)
val_generator = train_datagen.flow_from_directory(directory=Validate_data, target_size=(128,128),class_mode='categorical', batch_size=32)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = train_datagen.flow_from_directory(directory=Test_data,target_size=(128,128),class_mode='categorical',batch_size=32)

Found 10597 images belonging to 2 classes.
Found 1106 images belonging to 2 classes.
Found 1092 images belonging to 2 classes.


Defining the model 

In [48]:
#VGG19 model
model_file = '.h5'

if os.path.exists(model_file):
    model=load_model(model_file)
else:    
    vgg19 = VGG19(weights='imagenet',include_top=False,input_shape=(128,128,3))

    for layer in vgg19.layers:
        layer.trainable = False

    model = Sequential()
    model.add(vgg19)
    model.add(Flatten())
    model.add(Dense(2, activation = "sigmoid"))
    

80134624/80134624 [==============================] - 293s 4us/step


In [49]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten_7 (Flatten)         (None, 8192)              0         
                                                                 
 dense_14 (Dense)            (None, 2)                 16386     
                                                                 
Total params: 20,040,770
Trainable params: 16,386
Non-trainable params: 20,024,384
_________________________________________________________________


In [50]:
#Training
train_steps_per_epoch = ceil(len(train_data) / 32)
val_steps_per_epoch = ceil(len(val_generator) / 32)

history = model.fit_generator(generator=train_data,steps_per_epoch=train_steps_per_epoch,epochs=20,validation_data=val_generator, validation_steps=val_steps_per_epoch)

C:\Users\rana ahmed\AppData\Local\Temp\ipykernel_24868\1534015927.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_data,steps_per_epoch=train_steps_per_epoch,epochs=20,validation_data=val_generator, validation_steps=val_steps_per_epoch)


Epoch 1/20
11/11 [==============================] - 26s 2s/step - loss: 0.5173 - accuracy: 0.7472 - val_loss: 0.3757 - val_accuracy: 0.8594
Epoch 2/20
11/11 [==============================] - 24s 2s/step - loss: 0.3076 - accuracy: 0.8807 - val_loss: 0.2414 - val_accuracy: 0.9219
Epoch 3/20
11/11 [==============================] - 24s 2s/step - loss: 0.2330 - accuracy: 0.9233 - val_loss: 0.2022 - val_accuracy: 0.9062
Epoch 4/20
11/11 [==============================] - 24s 2s/step - loss: 0.1768 - accuracy: 0.9403 - val_loss: 0.1879 - val_accuracy: 0.9531
Epoch 5/20
11/11 [==============================] - 24s 2s/step - loss: 0.1792 - accuracy: 0.9347 - val_loss: 0.1516 - val_accuracy: 0.9375
Epoch 6/20
11/11 [==============================] - 24s 2s/step - loss: 0.1412 - accuracy: 0.9489 - val_loss: 0.1507 - val_accuracy: 0.9531
Epoch 7/20
11/11 [==============================] - 26s 2s/step - loss: 0.1496 - accuracy: 0.9545 - val_loss: 0.1807 - val_accuracy: 0.9531
Epoch 8/20
11/11 [==

In [51]:
model.evaluate(test_generator)

35/35 [==============================] - 65s 2s/step - loss: 0.0749 - accuracy: 0.9753


[0.0749480277299881, 0.9752747416496277]

In [52]:
model.save('MaskDetectionModel.h5')